# Math 428 - Homework for 04/03/19

## Problem 23.29, Part (a)
$$M=\int_{t_1}^{t_2} (9+4cos^2(0.4t))(5e^{-0.5t}+2e^{0.15t})dt$$

Determine the mass transported if $t_1=2$, $t_2=8$; using Romberg Integration with 0.1% tolerance.

In [1]:
import numpy as np
from pandas import *

In [2]:
# Compute the trapezoidal approximation of the function w/ provided subdivisions
def trapezoid_rule(f_x, x_start, x_end, num_divs):
    integral = f_x(x_start) + f_x(x_end)
    dx = (x_end - x_start) / num_divs
    for i in np.arange(1, num_divs):
        x_val = x_start + i * dx
        integral += 2 * f_x(x_val)
        
    return (integral * (x_end - x_start) / (2 * num_divs))

# Evaluates the integral of f(x) [x_start -> x_end] n times
# Doubles the number of trapezoidal segements used on each iteration
def integral_approx(f_x, x_start, x_end, num_levels):
    approx = []
    for i in range(num_levels):
        approx.append(trapezoid_rule(f_x, x_start, x_end, 2 ** i))
        
    return approx

# The Romberg Integration Approximation Algorithm
def romberg_integration(f_x, x_start, x_end, max_iters=10, actual_int=None, tol=None):
    # Grab the initial guesses from the trapezoidal approximation
    I = integral_approx(f_x, x_start, x_end, max_iters)
    # Convert the initial guesses into a properly sized square matrix
    I = np.append(I, np.zeros(max_iters*(max_iters-1))).reshape((max_iters, max_iters)).transpose()
    
    for i in range(1, max_iters):
        p4 = 1
        for j in range(1, i + 1):
            p4 *= 4
            I[i, j] = I[i, j-1] + (I[i, j-1] - I[i-1, j-1]) / (p4 - 1)
        if tol:
            if min(abs(I[:, j] - actual_int) / abs(actual_int)) < tol:
                # Roll the array up to the top
                I = np.array([np.roll(r, -c) for c, r in enumerate(I.transpose())]).transpose()
                return I[:i+1, :j+1]
            
    # Return the rolled array so the top right is the fixed value
    return np.array([np.roll(r, -c) for c, r in enumerate(I.transpose())]).transpose()

In [3]:
def f_x(x):
    return ((9 + 4*np.cos(0.4*x)**2) * (5*np.exp(-0.5*x) + 2*np.exp(0.15*x)))

a = romberg_integration(f_x, 2, 8, max_iters=10, actual_int=322.348, tol=0.001)
print ("Result from 0.1% tolerance:\n", DataFrame(a))
b = romberg_integration(f_x, 2, 8, max_iters=6)
print ("\nResult with only an iteration-only stopping criteria of 6:\n", DataFrame(b))

Result from 0.1% tolerance:
             0           1           2
0  411.260952  317.155295  322.595716
1  340.681709  322.255690    0.000000
2  326.862195    0.000000    0.000000

Result with only an iteration-only stopping criteria of 6:
             0           1           2           3           4           5
0  411.260952  317.155295  322.595716  322.345708  322.348374  322.348367
1  340.681709  322.255690  322.349614  322.348364  322.348367    0.000000
2  326.862195  322.343744  322.348383  322.348367    0.000000    0.000000
3  323.473357  322.348093  322.348367    0.000000    0.000000    0.000000
4  322.629409  322.348350    0.000000    0.000000    0.000000    0.000000
5  322.418615    0.000000    0.000000    0.000000    0.000000    0.000000


Thus, the best guess of the mass transported during this time, found using the Romberg Integration with a stopping tolerance of $0.1\%$, is $322.5957162204574$ mg. If the tolerance value was to be ignored, and the algorithm was allowed to compute more iterations than just the three necessary to reach $0.1\%$ tolerance, then a more accurate answer could be computed. As shown above with 6 iterations